# 한글 워드 크라우드

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import nltk
from wordcloud import WordCloud, STOPWORDS
from PIL import Image # 그림을 불러오는 패키지
from nltk.corpus import stopwords
import pickle
from matplotlib import font_manager, rc
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family = 'Malgun Gothic')
from konlpy.tag import Okt, Kkma, Komoran
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
#konlpy
okt = Okt()
kkma = Kkma()
kr = Komoran()

In [3]:
text = '열심히 코딩한 당신, 연휴에는 여행을 떠나 보세요.'

In [4]:
# 한글 형태소 분석기별 차이점
print(okt.morphs(text))
print(kkma.morphs(text))
print(kr.morphs(text))

['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '떠나', '보세요', '.']
['열심히', '코딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '떨', '나', '보', '세요', '.']
['열심히', '코', '딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '떠나', '아', '보', '시', '어요', '.']


In [5]:
# 품사 태깅 pos = part-of-speech tagging
# http://kkma.snu.ac.kr/documents/?doc=postag
print(okt.pos(text))
print(kkma.pos(text))
print(kr.pos(text))

[('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('떠나', 'Verb'), ('보세요', 'Verb'), ('.', 'Punctuation')]
[('열심히', 'MAG'), ('코딩', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('당신', 'NP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKM'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('떨', 'VV'), ('나', 'ECE'), ('보', 'VXV'), ('세요', 'EFN'), ('.', 'SF')]
[('열심히', 'MAG'), ('코', 'NNG'), ('딩', 'MAG'), ('하', 'XSV'), ('ㄴ', 'ETM'), ('당신', 'NNP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('떠나', 'VV'), ('아', 'EC'), ('보', 'VV'), ('시', 'EP'), ('어요', 'EF'), ('.', 'SF')]


# 명사만 추출

In [6]:
print(okt.nouns(text))
print(kkma.nouns(text))
print(kr.nouns(text))

['코딩', '당신', '연휴', '여행']
['코딩', '당신', '연휴', '여행']
['코', '당신', '연휴', '여행']


# 새로나온책 200권 tag 워드 크라우드 만들기

In [69]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [70]:
lis = soup.select('li.sub_book_list')
len(lis)

20

In [71]:
li = lis[0]

In [72]:
#서브페이지 링크 추출
book_url = li.find('a')['href']

In [73]:
res = requests.get(base_url + sub_url)
book_soup = BeautifulSoup(res.text, 'html.parser')

In [74]:
tags = soup.select('.tag_area > a')
len(tags)

0

In [64]:
tag_list = []
for i in tags:
    tag_list.append(i.get_text().strip)

In [65]:
print(tag_list)

[<built-in method strip of str object at 0x0000014AA79408A0>, <built-in method strip of str object at 0x0000014AA8665430>, <built-in method strip of str object at 0x0000014AA7425D50>]


In [66]:
tag_list = []
for page in tqdm(range(1, 11)):
    url = f'{base_url}{sub_url}?page={page}'
    res=requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('sub_book_list')
    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(sub_req.text, 'html.parser')
        tags = book_soup.select('.tag_area > a')
        for i in tags:
            tag_list.append(i.get_text().strip)

  0%|          | 0/10 [00:00<?, ?it/s]

In [67]:
len(tag_list)

0